In [ ]:

# import data preprocessing libraries
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle
import numpy as np
import os
import re

# load data
file = open('/True.txt', 'r', encoding='utf8')
lines = []
for i in file:
    lines.append(i)
file.close()

data =''
for i in lines:
    data = ' '.join(lines)

# clean data remove any unwanted chars
data = re.sub(r'[^\w\s]', '', data.strip().lower())
print(data[:10])

# tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
pickle.dump(tokenizer, open('/content/drive/MyDrive/Colab Notebooks/tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
print(sequence_data[:10])

vocabulary_size = len(tokenizer.word_index) + 1


# create sequences
sequence = []
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequence.append(words)

# divide the data into features (X) and labels (Y)
x = []
y = []
for i in sequence:
    x.append(i[0:3])
    y.append(i[3])

x = np.array(x)
y = np.array(y)
y = to_categorical(y, num_classes=vocabulary_size)


# build the model
model = Sequential()
model.add(Embedding(vocabulary_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocabulary_size, activation="softmax"))

model.summary()

# compile and train the model
checkPoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/True_model.keras',
                             monitor='loss', verbose=1, save_best_only=True)

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))
model.fit(x, y, epochs=50, batch_size=64, callbacks=[checkPoint])
